# À propos du dataset : 

### Description :
Cet ensemble de données est le résultat de la fusion de plusieurs sources de données distinctes, fournissant des informations pertinentes sur différentes années, villes, indicateurs socio-économiques, et variables politiques. Ces données sont recueillies sur une période de plusieurs années, offrant ainsi une vue globale et historique des caractéristiques de différentes régions.
### Colonnes :
Année : L'année à laquelle les données ont été enregistrées.
Villes : Le nom de la ville concernée par les données.
Nb_de_population : Le nombre total de population dans la ville.
Population : La population totale de la ville.
Revenus_activité : Les revenus issus de l'activité économique dans la ville.
Impôts : Les impôts collectés dans la ville.
PIB_par_habitant : Le produit intérieur brut (PIB) par habitant dans la ville.
Nb_faits_delinquance : Le nombre total de faits de délinquance enregistrés.
Orientation_politique : L'orientation politique dominante dans la ville.
Nb_voix : Le nombre de voix ou votes enregistrés pour cette orientation politique.

### nombre de lignes : 222

### Sources de données : 
Ce dataset a été créé en combinant plusieurs ensembles de données provenant de sources diverses, afin de fournir une vue d'ensemble complète des données socio-économiques et politiques à travers le temps et l'espace.




In [42]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler


In [43]:
election_data=pd.read_excel('dataset_version_final.xlsx')

In [4]:

election_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   annee                  221 non-null    int64 
 1   villes                 221 non-null    object
 2   nb_de                  221 non-null    int64 
 3   population             221 non-null    int64 
 4   revenus_activite       221 non-null    int64 
 5   Impots                 221 non-null    int64 
 6   pib_par_habitant       221 non-null    int64 
 7   nb_faits_delinquance   221 non-null    int64 
 8   orientation_politique  221 non-null    object
 9   nb_voix                221 non-null    int64 
dtypes: int64(8), object(2)
memory usage: 17.4+ KB


In [5]:
election_data

,annee,villes,nb_de,population,revenus_activite,Impots,pib_par_habitant,nb_faits_delinquance,orientation_politique,nb_voix
0,2002,Bordeaux,351910,332708,13624,1875,22281,194757,Centre gauche,8321
1,2002,Lille,439520,61375,13126,1761,20399,259523,Centre gauche,7862
2,2002,Lyon,406490,199114,15112,2108,24702,310000,Centre gauche,17868
3,2002,Marseille,344430,409558,13989,2225,23416,274260,Centre gauche,19609
4,2002,Nantes,207510,116395,13821,1833,22954,128747,Centre gauche,12678
...,...,...,...,...,...,...,...,...,...,...
216,2022,Marseille,544510,657316,19856,3938,35273,266865,Gauche,7042
217,2022,Nantes,344950,242669,18775,3248,32838,135871,Gauche,2639
218,2022,Rennes,292530,169395,18311,3177,31820,101597,Gauche,1959
219,2022,Strasbourg,493760,215992,19396,3237,30023,181215,Gauche,1256


# Encodage : 
Il s'agit de convertir les velrus de les colonnes  "Orientation Politique et  Villes"  de données textuelles en une forme numérique appropriée pour l'analyse ou la modélisation.

En suivant le mapping ci-dessous :

oritentation_politique : {
    "Extrème gauche" : 1
    "Extrème droite":2
    "Droite":3
    "Gauche":4
    "Centre":5
    "Centre gauche":6
    "Centre droite":7
}

villes : {
   "Bordeaux":33
    "Lille":59
    "Lyon":69
    "Marseille":13
    "Nantes":44
    "Rennes":35
    "Strasbourg":67
    "Toulouse":31
}




In [46]:
correspondance_orientation = {
    "Extrème gauche": 1,
    "Extrème droite": 2,
    "Droite": 3,
    "Gauche": 4,
    "Centre": 5,
    "Centre gauche": 6,
    "Centre droite": 7
}

correspondance_villes = {
    "Bordeaux": 33,
    "Lille": 59,
    "Lyon": 69,
    "Marseille": 13,
    "Nantes": 44,
    "Rennes": 35,
    "Strasbourg": 67,
    "Toulouse": 31
}

# Utiliser la fonction replace pour modifier les colonnes en fonction des correspondances
election_data['orientation_politique'] = election_data['orientation_politique'].replace(correspondance_orientation)
election_data['villes'] = election_data['villes'].replace(correspondance_villes)

Obtenir des mesures statistiques sur les données


In [47]:
election_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   annee                  221 non-null    int64
 1   villes                 221 non-null    int64
 2   nb_de                  221 non-null    int64
 3   population             221 non-null    int64
 4   revenus_activite       221 non-null    int64
 5   Impots                 221 non-null    int64
 6   pib_par_habitant       221 non-null    int64
 7   nb_faits_delinquance   221 non-null    int64
 8   orientation_politique  221 non-null    int64
 9   nb_voix                221 non-null    int64
dtypes: int64(10)
memory usage: 17.4 KB


# Apprentissage Supervisé

SVC Model Training

In [39]:

# Séparez les caractéristiques (X) de la cible (y)
X = election_data.drop('orientation_politique', axis=1)
y = election_data['orientation_politique']

# Séparez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisez les données en utilisant Min-Max Scaling
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Initialisez le modèle SVM
svm_model = SVC(kernel='linear')

# Entraînez le modèle sur l'ensemble d'entraînement normalisé
svm_model.fit(X_train_normalized, y_train)

# Faites des prédictions sur l'ensemble de test normalisé
y_pred = svm_model.predict(X_test_normalized)

# Évaluez les performances du modèle en utilisant la précision
accuracy = accuracy_score(y_test, y_pred)
print(f'Précision du modèle SVM : {accuracy * 100:.2f}%')


Précision du modèle SVM : 13.33%
